# data

In [1]:
%load_ext autoreload
%autoreload 2
import importlib
from importlib import reload  
  
import monotonic
import utils
from train import execute_model_evaluation
from train_config import FlatACLConfig
import train_config
import data_manager
from data_manager import DatasetManager
import metrics
from dataclasses import replace

reload(utils)
reload(monotonic)
reload(train_config)


%load_ext autoreload
%autoreload 2 
data_mgr = DatasetManager(device='cuda')
 
data_mgr.load_csv('ettm2', './ettm2.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Dataset: ettm2 (csv)
Shape: torch.Size([69680, 7])
Channels: 7
Length: 69680
Source: ./ettm2.csv

Sample data (first 2 rows):
tensor([[41.1300, 12.4810, 36.5360,  9.3550,  4.4240,  1.3110, 38.6620],
        [39.6220, 11.3090, 35.5440,  8.5510,  3.2090,  1.2580, 38.2230]])


# Seq=336

### EigenACL

#### pred=96

##### huber

In [2]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
global_std.shape: torch.Size([7])
Global Std for ettm2: tensor([10.2434,  6.0312, 13.0618,  4.3690,  6.1544,  6.0135, 11.8865],
       device='cuda:0')
Train set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 378
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 378
Validation Batches: 52
Test Batches: 106

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 34.9086, mae: 2.9131, huber: 2.4963, swd: 24.9905, ept: 81

In [20]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.enable_magnitudes = [False, True]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 378
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 378
Validation Batches: 52
Test Batches: 106

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 31.6610, mae: 2.8255, huber: 2.4096, swd: 23.2325, target_std: 20.3532
Epoch [1/50], Val Losses: mse: 15.6386, mae: 2.4859, huber: 2.0822, swd: 9.5497, target_std: 20.5735
Epoch [1/50], Test Losses: mse: 11.364

In [39]:
importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=True,
    householder_reflects_latent = 4,
    householder_reflects_data = 8,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 378
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 378
Validation Batches: 52
Test Batches: 106

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 31.1372, mae: 2.8134, huber: 2.3973, swd: 21.9180, target_std: 20.3536
Epoch [1/50], Val Losses: mse: 14.0418, mae: 2.3357, huber: 1.9325, swd: 7.5180, target_std: 20.5735
Epoch [1/50], Test Losses: mse: 10.712

In [42]:
importlib.reload(monotonic)
importlib.reload(train_config) 
import torch.nn as nn
cfg = train_config.FlatACLConfig(  
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=True,
    householder_reflects_latent = 4,
    householder_reflects_data = 8,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_hidden_layers = [nn.LogSigmoid, nn.Mish]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_hidden_layers = [nn.LogSigmoid, nn.Mish]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_hidden_layers = [nn.LogSigmoid, nn.Mish]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_hidden_layers = [nn.LogSigmoid, nn.Mish]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_hidden_layers = [nn.LogSigmoid, nn.Mish]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)



Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 378
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 378
Validation Batches: 52
Test Batches: 106

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 31.5703, mae: 2.8403, huber: 2.4277, swd: 20.9222, target_std: 20.3536
Epoch [1/50], Val Losses: mse: 13.3048, mae: 2.2805, huber: 1.8886, swd: 7.2631, target_std: 20.5735
Epoch [1/50], Test Losses: mse: 10.013

#### pred=196

##### huber

In [3]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
global_std.shape: torch.Size([7])
Global Std for ettm2: tensor([10.2434,  6.0312, 13.0618,  4.3690,  6.1544,  6.0135, 11.8865],
       device='cuda:0')
Train set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 377
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 196, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 377
Validation Batches: 51
Test Batches: 105

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 38.4921, mae: 3.1895, huber: 2.7680, swd: 26.4039, ept

In [34]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.enable_magnitudes = [False, True]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 377
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 196, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 377
Validation Batches: 51
Test Batches: 105

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 37.2775, mae: 3.0779, huber: 2.6587, swd: 26.6849, target_std: 20.3580
Epoch [1/50], Val Losses: mse: 17.6471, mae: 2.6186, huber: 2.2082, swd: 9.0289, target_std: 20.5630
Epoch [1/50], Test Losses: mse: 12

#### pred=336

##### huber

In [4]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
global_std.shape: torch.Size([7])
Global Std for ettm2: tensor([10.2434,  6.0312, 13.0618,  4.3690,  6.1544,  6.0135, 11.8865],
       device='cuda:0')
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 42.6391, mae: 3.4097, huber: 2.9854, swd: 27.8345, ept

##### model I

In [24]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
    

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 42.6391, mae: 3.4097, huber: 2.9854, swd: 27.8345, target_std: 20.3626
Epoch [1/50], Val Losses: mse: 23.8987, mae: 3.1318, huber: 2.7119, swd: 9.1935, target_std: 20.5432
Epoch [1/50], Test Losses: mse: 17

##### model II

In [25]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=True,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=True, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,  nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 35.1773, mae: 3.1092, huber: 2.6877, swd: 18.0032, target_std: 20.3629
Epoch [1/50], Val Losses: mse: 25.6371, mae: 3.1818, huber: 2.7641, swd: 12.2080, target_std: 20.5432
Epoch [1/50], Test Losses: mse: 1

##### model III

In [26]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 4,
    householder_reflects_data = 8,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,  nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 41.7774, mae: 3.3723, huber: 2.9479, swd: 27.4229, target_std: 20.3625
Epoch [1/50], Val Losses: mse: 23.2774, mae: 3.0707, huber: 2.6497, swd: 9.2939, target_std: 20.5432
Epoch [1/50], Test Losses: mse: 17

#### pred=720

##### huber

In [5]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
global_std.shape: torch.Size([7])
Global Std for ettm2: tensor([10.2434,  6.0312, 13.0618,  4.3690,  6.1544,  6.0135, 11.8865],
       device='cuda:0')
Train set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 373
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 720, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 373
Validation Batches: 47
Test Batches: 101

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 49.1809, mae: 3.7493, huber: 3.3193, swd: 30.7613, ept

In [36]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['ettm2']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.enable_magnitudes = [False, True]
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 373
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 720, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 373
Validation Batches: 47
Test Batches: 101

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 54.7940, mae: 3.9060, huber: 3.4764, swd: 36.3485, target_std: 20.3716
Epoch [1/50], Val Losses: mse: 27.4639, mae: 3.4771, huber: 3.0505, swd: 9.8745, target_std: 20.5636
Epoch [1/50], Test Losses: mse: 19

### Timemixer

#### pred=96

In [29]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 378
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 378
Validation Batches: 52
Test Batches: 106

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 11.4886, mae: 1.8500, huber: 1.4600, swd: 6.4130, target_std: 20.3534
Epoch [1/50], Val Losses: mse: 12.0195, mae: 2.0837, huber: 1.6888, swd: 6.45

#### pred=196

In [31]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 377
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 196, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 377
Validation Batches: 51
Test Batches: 105

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 16.2482, mae: 2.1641, huber: 1.7665, swd: 9.0275, target_std: 20.3579
Epoch [1/50], Val Losses: mse: 17.1636, mae: 2.5089, huber: 2.1041, swd: 

#### pred=336

##### huber

In [32]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 21.3876, mae: 2.4607, huber: 2.0573, swd: 11.4909, target_std: 20.3628
Epoch [1/50], Val Losses: mse: 20.8653, mae: 2.8194, huber: 2.4075, swd:

##### huber + 0.1SWD

In [18]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.1, 0.0]
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 21.6828, mae: 2.5282, huber: 2.1219, swd: 11.0287, target_std: 20.3628
Epoch [1/50], Val Losses: mse: 20.6632, mae: 2.8228, huber: 2.4092, swd:

##### huber + 0.5SWD

In [9]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.5, 0.0]
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 25.2914, mae: 2.7466, huber: 2.3351, swd: 10.6852, target_std: 20.3628
Epoch [1/50], Val Losses: mse: 23.0500, mae: 2.9698, huber: 2.5517, swd:

##### huber + 1.5 SWD

In [21]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 1.5, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 1.5, 0.0]
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 30.6418, mae: 3.0377, huber: 2.6204, swd: 10.3998, target_std: 20.3628
Epoch [1/50], Val Losses: mse: 27.8412, mae: 3.2507, huber: 2.8267, swd:

##### MSE

In [ ]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.0, 0.0]
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


##### MSE + 0.1 SWD

In [19]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.1, 0.0]
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 21.1148, mae: 2.5399, huber: 2.1319, swd: 11.1521, target_std: 20.3628
Epoch [1/50], Val Losses: mse: 21.1792, mae: 2.8622, huber: 2.4471, swd:

##### MSE + 0.5 SWD

In [10]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.5, 0.0]
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 21.2849, mae: 2.5525, huber: 2.1440, swd: 11.1402, target_std: 20.3628
Epoch [1/50], Val Losses: mse: 21.1260, mae: 2.8614, huber: 2.4458, swd:

##### MSE + 1.5 SWD

In [20]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 1.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 1.5, 0.0]
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 21.6208, mae: 2.5792, huber: 2.1698, swd: 11.1055, target_std: 20.3628
Epoch [1/50], Val Losses: mse: 21.1509, mae: 2.8728, huber: 2.4557, swd:

#### pred=720

In [33]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 373
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 720, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 373
Validation Batches: 47
Test Batches: 101

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 29.1677, mae: 2.8764, huber: 2.4645, swd: 14.7016, target_std: 20.3717
Epoch [1/50], Val Losses: mse: 24.4334, mae: 3.1730, huber: 2.7519, swd:

### PatchTST

#### pred=96

In [25]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 378
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 378
Validation Batches: 52
Test Batches: 106

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 12.8992, mae: 1.9811, huber: 1.5850, swd: 6.4544, target_std: 20.3533
Epoch [1/50], Val Losses: mse: 12.0627, mae: 2.1191, huber: 1.7197, swd: 6.32

#### pred=196

In [26]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 377
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 196, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 377
Validation Batches: 51
Test Batches: 105

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 17.6480, mae: 2.2844, huber: 1.8813, swd: 9.1927, target_std: 20.3579
Epoch [1/50], Val Losses: mse: 16.4263, mae: 2.4967, huber: 2.0876, swd: 

#### 336-336

##### huber

In [27]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 22.4797, mae: 2.5568, huber: 2.1489, swd: 11.4615, target_std: 20.3630
Epoch [1/50], Val Losses: mse: 19.6520, mae: 2.7810, huber: 2.3636, swd:

##### huber + 0.1 SWD

In [17]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 23.8867, mae: 2.6977, huber: 2.2851, swd: 11.7808, target_std: 20.3630
Epoch [1/50], Val Losses: mse: 20.1794, mae: 2.8652, huber: 2.4436, swd:

##### huber + 0.5 SWD

In [4]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 29.8690, mae: 3.0650, huber: 2.6441, swd: 12.2708, target_std: 20.3630
Epoch [1/50], Val Losses: mse: 22.0883, mae: 2.9850, huber: 2.5612, swd:

##### huber + 1.5 SWD

In [22]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [0.0, 0.0, 1.0, 1.5, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 1.5, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================



[autoreload of data_manager failed: Traceback (most recent call last):
  File "c:\Users\qilin\miniconda3\envs\ww\Lib\site-packages\IPython\extensions\autoreload.py", line 283, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\qilin\miniconda3\envs\ww\Lib\site-packages\IPython\extensions\autoreload.py", line 483, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "c:\Users\qilin\miniconda3\envs\ww\Lib\importlib\__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1133, in get_code
  File "<frozen importlib._bootstrap_external>", line 1063, in source_to_code
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "c:\proj\DL_notebook\study\data_manager.py", line 196
    def load_csv(self, name, path, date_column

Epoch [1/50], Train Losses: mse: 35.5547, mae: 3.3709, huber: 2.9451, swd: 12.3772, target_std: 20.3630
Epoch [1/50], Val Losses: mse: 32.1230, mae: 3.5945, huber: 3.1584, swd: 8.8195, target_std: 20.5432
Epoch [1/50], Test Losses: mse: 23.0310, mae: 3.0056, huber: 2.5723, swd: 7.2388, target_std: 18.3439
  Epoch 1 composite train-obj: 21.510884
        Val objective improved inf → 16.3876, saving checkpoint.
Epoch [2/50], Train Losses: mse: 29.9709, mae: 3.0407, huber: 2.6181, swd: 10.2398, target_std: 20.3628
Epoch [2/50], Val Losses: mse: 23.5775, mae: 3.0936, huber: 2.6649, swd: 8.5048, target_std: 20.5432
Epoch [2/50], Test Losses: mse: 18.0588, mae: 2.6431, huber: 2.2182, swd: 7.7251, target_std: 18.3439
  Epoch 2 composite train-obj: 17.977802
        Val objective improved 16.3876 → 15.4221, saving checkpoint.
Epoch [3/50], Train Losses: mse: 26.9337, mae: 2.8324, huber: 2.4140, swd: 9.2852, target_std: 20.3630
Epoch [3/50], Val Losses: mse: 24.0871, mae: 3.1493, huber: 2.7180,

##### MSE

In [5]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 22.9891, mae: 2.6711, huber: 2.2567, swd: 11.6396, target_std: 20.3630
Epoch [1/50], Val Losses: mse: 20.1748, mae: 2.8810, huber: 2.4577, swd:

##### MSE + 0.1SWD

In [23]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 23.0066, mae: 2.6772, huber: 2.2627, swd: 11.6080, target_std: 20.3630
Epoch [1/50], Val Losses: mse: 20.7187, mae: 2.9297, huber: 2.5041, swd:

##### MSE + 0.5SWD

In [6]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 23.3654, mae: 2.7020, huber: 2.2871, swd: 11.8234, target_std: 20.3630
Epoch [1/50], Val Losses: mse: 21.7175, mae: 2.9895, huber: 2.5629, swd:

##### MSE + 1.5SWD

In [7]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [1.0, 0.0, 0.0, 1.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 1.5, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 24.2731, mae: 2.7659, huber: 2.3497, swd: 12.1862, target_std: 20.3630
Epoch [1/50], Val Losses: mse: 21.4858, mae: 2.9763, huber: 2.5499, swd:

In [ ]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

#### pred=720

In [28]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['ettm2']['channels'],
    enc_in=data_mgr.datasets['ettm2']['channels'],
    dec_in=data_mgr.datasets['ettm2']['channels'],
    c_out=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 373
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 720, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 373
Validation Batches: 47
Test Batches: 101

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 30.1986, mae: 2.9684, huber: 2.5525, swd: 14.7643, target_std: 20.3719
Epoch [1/50], Val Losses: mse: 24.6203, mae: 3.2236, huber: 2.7947, swd:

### DLinear

#### pred=96

In [21]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 378
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 378
Validation Batches: 52
Test Batches: 106

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 22.9063, mae: 2.3735, huber: 1.9679, swd: 10.2081, target_std: 20.3538
Epoch [1/50], Val Losses: mse: 13.4917, mae: 2.1931, huber: 1.7944, swd: 7.6

#### pred=196

In [22]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([196, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 377
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 196, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 377
Validation Batches: 51
Test Batches: 105

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 28.4604, mae: 2.6780, huber: 2.2667, swd: 13.3442, target_std: 20.3578
Epoch [1/50], Val Losses: mse: 18.7906, mae: 2.6063, huber: 2.1985, swd:

#### pred=336

##### huber

In [23]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 32.0404, mae: 2.9122, huber: 2.4961, swd: 14.8301, target_std: 20.3632
Epoch [1/50], Val Losses: mse: 22.2878, mae: 2.9277, huber: 2.5117, swd:

##### huber + 0.1SWD

In [14]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 54.1800, mae: 3.2874, huber: 2.8678, swd: 13.7802, target_std: 20.3632
Epoch [1/50], Val Losses: mse: 23.0501, mae: 3.0088, huber: 2.5879, swd:

##### huber + 0.5SWD

In [11]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 137.0969, mae: 4.6951, huber: 4.2661, swd: 13.8548, target_std: 20.3632
Epoch [1/50], Val Losses: mse: 27.6261, mae: 3.2870, huber: 2.8584, swd

##### MSE 

In [12]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 34.1505, mae: 3.0724, huber: 2.6500, swd: 15.5327, target_std: 20.3632
Epoch [1/50], Val Losses: mse: 22.0402, mae: 2.9422, huber: 2.5213, swd:

##### MSE + 0.1 SWD

In [15]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 34.1893, mae: 3.0795, huber: 2.6568, swd: 15.2033, target_std: 20.3632
Epoch [1/50], Val Losses: mse: 22.1322, mae: 2.9479, huber: 2.5267, swd:

##### MSE + 0.5SWD

In [13]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 34.5239, mae: 3.1058, huber: 2.6823, swd: 14.7230, target_std: 20.3632
Epoch [1/50], Val Losses: mse: 22.2841, mae: 2.9601, huber: 2.5380, swd:

##### MSE + 1.5SWD

In [16]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 1.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 1.5, 0.0],
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 376
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 376
Validation Batches: 50
Test Batches: 104

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 35.6532, mae: 3.1501, huber: 2.7247, swd: 14.2689, target_std: 20.3632
Epoch [1/50], Val Losses: mse: 22.9741, mae: 3.0216, huber: 2.5961, swd:

#### pred=720

In [24]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['ettm2']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('ettm2', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([48776, 7])
Shape of validation data: torch.Size([6968, 7])
Shape of testing data: torch.Size([13936, 7])
Train set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Validation set sample shapes: torch.Size([336, 7]), torch.Size([720, 7])
Test set data shapes: torch.Size([13936, 7]), torch.Size([13936, 7])
Number of batches in train_loader: 373
Batch 0: Data shape torch.Size([128, 336, 7]), Target shape torch.Size([128, 720, 7])

Data Preparation: ettm2
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 373
Validation Batches: 47
Test Batches: 101

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 39.9552, mae: 3.3089, huber: 2.8861, swd: 17.9193, target_std: 20.3713
Epoch [1/50], Val Losses: mse: 26.9272, mae: 3.3462, huber: 2.9194, swd: